### Citibike Analysis

In [1]:
import os
from dotenv import load_dotenv

from google.cloud import bigquery
from google.api_core.exceptions import NotFound, BadRequest


import numpy as np


In [2]:
load_dotenv()

True

In [3]:
client = bigquery.Client()

bq_project_id = os.getenv('GCP_PROJECT_ID')
bq_dataset_name = os.getenv('BQ_DATASET')
bq_table_name = os.getenv('BQ_TABLE')

table_red = f'{bq_project_id}.{bq_dataset_name}.{bq_table_name}'
cleaned_table_ref = f'{bq_project_id}.{bq_dataset_name}.{bq_table_name}_cleaned'

In [4]:
query = f"""
    SELECT *
    FROM `{bq_dataset_name}.{bq_table_name}`
    LIMIT 10
"""

df = client.query(query).to_dataframe()
df.head()

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,cleaned_start_station_id,cleaned_start_station_name
0,3F97AB2320337E97,classic_bike,2023-12-31 19:17:21.067000+00:00,2024-01-01 20:16:59.127000+00:00,W 37 St & Broadway,6441.06,None,nan,40.752271,-73.987706,NaN,NaN,casual,None,None
1,36D0948945B60A14,classic_bike,2023-12-31 06:25:55.501000+00:00,2024-01-01 07:25:47.279000+00:00,W 37 St & Broadway,6441.06,None,nan,40.752271,-73.987706,NaN,NaN,casual,None,None
2,B82F865E66B13DA3,classic_bike,2023-12-31 14:50:10.465000+00:00,2024-01-01 15:50:04.927000+00:00,Central Park West & W 76 St,7253.04,None,nan,40.778968,-73.973747,NaN,NaN,casual,None,None
3,068B974DABC0B2B8,classic_bike,2023-12-31 12:53:15.228000+00:00,2024-01-01 13:52:56.073000+00:00,Soissons Landing,4590.01,None,nan,40.692317,-74.014866,NaN,NaN,casual,None,None
4,9FF2EF018137DE33,classic_bike,2023-12-31 12:11:27.022000+00:00,2024-01-01 13:11:20.711000+00:00,Bergen St & 4 Ave,4322.06,None,nan,40.682564,-73.979898,NaN,NaN,casual,None,None


In [5]:
df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'cleaned_start_station_id',
       'cleaned_start_station_name'],
      dtype='object')

In [6]:
df.dtypes

ride_id                                    object
rideable_type                              object
started_at                    datetime64[us, UTC]
ended_at                      datetime64[us, UTC]
start_station_name                         object
start_station_id                           object
end_station_name                           object
end_station_id                             object
start_lat                                 float64
start_lng                                 float64
end_lat                                   float64
end_lng                                   float64
member_casual                              object
cleaned_start_station_id                   object
cleaned_start_station_name                 object
dtype: object

In [7]:
query = f"""
    SELECT 
        COUNT(*) - COUNT(ride_id) as ride_id_null,
        COUNT(*) - COUNT(rideable_type) as rideable_type_null,
        COUNT(*) - COUNT(started_at) as started_at_null,
        COUNT(*) - COUNT(ended_at) as ended_at_null,
        COUNT(*) - COUNT(start_station_name) as start_station_name_null,
        COUNT(*) - COUNT(start_station_id) as start_station_idnull,
        COUNT(*) - COUNT(end_station_name) as end_station_name_null,
        COUNT(*) - COUNT(end_station_id) as end_station_id_null,
        COUNT(*) - COUNT(start_lat) as start_lat_null,
        COUNT(*) - COUNT(start_lng) as start_lng_null,
        COUNT(*) - COUNT(end_lat) as end_lat_null,
        COUNT(*) - COUNT(end_lng) as end_lng_null,
        COUNT(*) - COUNT(member_casual) as member_casual_null
    FROM `{bq_dataset_name}.{bq_table_name}`
"""

df = client.query(query).to_dataframe()
df.T

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,0
ride_id_null,0
rideable_type_null,0
started_at_null,0
ended_at_null,0
start_station_name_null,35253
start_station_idnull,0
end_station_name_null,152745
end_station_id_null,0
start_lat_null,0
start_lng_null,0


In [8]:
query = f"""
    SELECT start_station_id, COUNT(*)
    FROM `{bq_dataset_name}.{bq_table_name}`
    WHERE start_station_id IS NULL OR SAFE_CAST(start_station_id AS FLOAT64) IS NULL
    GROUP BY start_station_id;
"""

df = client.query(query).to_dataframe()
print(df['start_station_id'].astype(str).tolist())

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


['HB602', 'JC104', 'JC102', 'JC018', 'JC080', 'HB503', 'JC002', 'HB301', 'JC115', 'JC094', 'JC076', 'JC109', 'HB105', 'JC055', 'HB408', 'JC013', 'JC052', 'JC074', 'HB302', 'LA Metro Demo 2', 'JC008', 'Lab - NYC - Monolith', '5308.04_', 'HB407', 'JC032', 'HB305', 'HB202', 'Lab - NYC', 'JC066', 'JC098', 'HB603', 'JC024', 'HB502', 'HB303', 'LA Metro Demo 1', 'HB404', 'HB506', 'JC093', 'HB601', 'JC116', 'JC078', '6173.08_Pillar', 'HB203', 'JC027', '6247.06_Pillar', 'HB401', 'JC023', 'SYS033', 'HB201', '190 Morgan', 'JC072', 'HB102', '6569.09_', 'JC038', 'HB501', 'JC019', 'HB505', 'JC014', 'HB304', 'Shop Morgan ', 'HB611', 'JC103', 'JC099', 'JC020', 'HB101', 'HB103', 'JC105', 'JC022', 'JC003', 'HB609', 'SYS025', 'JC009', 'SYS016', 'SYS038', 'JC081', 'JC084', 'JC075']


In [60]:
# Regex patterns to exclude
regex_patterns = [
    r'^[A-Za-z]{2}\d{3}$',  # Two letters + 3 digits
    r'^SYS\d{3}$',          # SYS + 3 digits
    r'(?i)Demo',            # Contains "Demo"
    r'(?i)Lab - NYC',       # Contains "Lab - NYC"
    r'(?i)Morgan'           # Contains "Morgan"
]

# Combine patterns using alternation (OR)
combined_pattern = '|'.join(regex_patterns)

# Create the query excluding the combined regex patterns
query = f"""
CREATE OR REPLACE TABLE `{bq_dataset_name}.{bq_table_name}_cleaned`
PARTITION BY DATE_TRUNC(started_at, MONTH)
CLUSTER BY member_casual, rideable_type

 AS

  SELECT * EXCEPT(start_station_id, start_station_name),
  -- Clean start_station_id
  CASE
    WHEN REGEXP_CONTAINS(start_station_id, r'^\\d+\\.\\d+_$') THEN REPLACE(start_station_id, '_', '')
    WHEN CONTAINS_SUBSTR(start_station_id, 'Pillar') THEN REGEXP_REPLACE(start_station_id, r'_Pillar$', '')
    ELSE start_station_id
  END AS start_station_id,

  -- Clean start_station_name only if ID contains 'Pillar'
  CASE
    WHEN CONTAINS_SUBSTR(start_station_id, 'Pillar') THEN TRIM(REGEXP_REPLACE(start_station_name, r'(?i)\s*Pillar\s*', ' '))
    ELSE TRIM(REGEXP_REPLACE(start_station_name, r'[\s\u00A0\u200B]+', ' ')) 
  END AS start_station_name

FROM `{bq_dataset_name}.{bq_table_name}`
WHERE NOT REGEXP_CONTAINS(start_station_id, r"{combined_pattern}")
"""

job = client.query(query)
job.result()


<>:37: SyntaxWarning: invalid escape sequence '\s'
<>:37: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_15462/3094358261.py:37: SyntaxWarning: invalid escape sequence '\s'
  """


In [61]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM `{cleaned_table_ref}`
    WHERE start_station_id IS NULL OR SAFE_CAST(start_station_id AS FLOAT64) IS NULL
    GROUP BY start_station_name, start_station_id;
"""

df = client.query(query).to_dataframe()
df

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_


In [62]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM {cleaned_table_ref}
    GROUP BY start_station_name, start_station_id
"""

df = client.query(query).to_dataframe()
df[df['start_station_id'].duplicated(keep=False)].sort_values(by='start_station_id')

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_
26,Eastern Pkwy & St Marks Ave,3982.01,1169
570,Eastern Pkwy\t& St Marks Ave,3982.01,1557
1808,Bridge St & Front St,4968.03,19105
1744,Bridge St & Water St,4968.03,26853
573,Morton St & Washington St,5772.05,30999
773,Morton St & Greenwich St,5772.05,39300
2145,34th Ave & Vernon Blvd,6873.01,12397
435,34 Ave & Vernon Blvd,6873.01,3864
453,Central Park West & W 68 St,7079.06,82351
2187,Central Park W & W 68 St,7079.06,26746


In [63]:
query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_id = CAST(FORMAT('%.2f', CAST(start_station_id AS FLOAT64)) AS STRING)
    WHERE TRUE
"""
job = client.query(query)
job.result()

In [64]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM {cleaned_table_ref}
    GROUP BY start_station_name, start_station_id
"""

df = client.query(query).to_dataframe()
df[df['start_station_id'].duplicated(keep=False)].sort_values(by='start_station_id')

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_
66,Eastern Pkwy & St Marks Ave,3982.01,1169
1155,Eastern Pkwy\t& St Marks Ave,3982.01,1557
1547,Bridge St & Water St,4968.03,26853
943,Bridge St & Front St,4968.03,19105
1119,Morton St & Greenwich St,5772.05,39300
363,Morton St & Washington St,5772.05,30999
1946,34 Ave & Vernon Blvd,6873.01,3864
1566,34th Ave & Vernon Blvd,6873.01,12397
1014,Central Park W & W 68 St,7079.06,26746
254,Central Park West & W 68 St,7079.06,82351


In [65]:
convert_names = {
    'West': 'W',
    'Fort': 'Ft',
    'Av': 'Ave',
    'Ichan Stadium': 'Icahn Stadium'
}

sql_expr = "start_station_name"
for old, new in convert_names.items():
    sql_expr = f"REGEXP_REPLACE({sql_expr}, r'\\b{old}\\b', '{new}')"

query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_name = {sql_expr}
    WHERE TRUE
"""

job = client.query(query)
job.result()

In [66]:
pattern = r'\\t'

query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_name = REGEXP_REPLACE(start_station_name, r'{pattern}', ' ')
    WHERE REGEXP_CONTAINS(start_station_name, r'{pattern}')
"""

job = client.query(query)
job.result()

In [67]:
pattern = r'(\d+)(st|nd|rd|th)\b'
replace = r'\1'

query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_id = REGEXP_REPLACE(start_station_name, r'{pattern}', r'{replace}')
    WHERE REGEXP_CONTAINS(start_station_name, r'{pattern}')
"""

job = client.query(query)
job.result()

In [68]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM {cleaned_table_ref}
    GROUP BY start_station_name, start_station_id
"""

df = client.query(query).to_dataframe()
df[df['start_station_id'].duplicated(keep=False)].sort_values(by='start_station_id')

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_
846,Bridge St & Front St,4968.03,19105
1434,Bridge St & Water St,4968.03,26853
443,Morton St & Greenwich St,5772.05,39300
1401,Morton St & Washington St,5772.05,30999
1740,3 Ave & E 82 St,7154.10,41574
2217,3 Ave & E 81 St,7154.10,21819


In [76]:
convert_names = {
    'Bridge St & Water St': 'Bridge St & Front St',
    'Morton St & Washington St': 'Morton St & Greenwich St',
    '3 Ave & E 81 St': '3 Ave & E 82 St'
}

case_statements = []
for old, new in convert_names.items():
    case_statements.append(f"WHEN start_station_name = '{old}' THEN '{new}'")

query = f"""
    UPDATE {cleaned_table_ref}
    SET start_station_name = CASE {' '.join(case_statements)} ELSE start_station_name END
    WHERE start_station_name IN ({', '.join(f"'{old}'" for old in convert_names.keys())})
"""

job = client.query(query)
job.result()

In [77]:
query = f"""
    SELECT start_station_name, start_station_id, COUNT(*)
    FROM {cleaned_table_ref}
    GROUP BY start_station_name, start_station_id
"""

df = client.query(query).to_dataframe()
df[df['start_station_id'].duplicated(keep=False)].sort_values(by='start_station_id')

/home/olshe/pers/eu-climate-economic-resilience/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1957: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


,start_station_name,start_station_id,f0_
